In [ ]:
%run "0_Workspace_setup.ipynb"

from NHM_helpers.NHM_hydrofabric import make_hf_map_elements, create_poi_df
from NHM_helpers.map_template import make_hf_map
import webbrowser


(   hru_gdf,
    hru_txt,
    hru_cal_level_txt,
    seg_gdf,
    seg_txt,
    nwis_gages_aoi,
    poi_df,
    gages_df,
    gages_txt,
    gages_txt_nb2,
    HW_basins_gdf,
    HW_basins,
) = make_hf_map_elements(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    control_file_name,
    nwis_gages_file,
    gages_file,
    default_gages_file,
    nhru_params,
    nhru_nmonths_params,
    nwis_gage_nobs_min,
)

con.print(
    f"{workspace_txt}\n",
    f"\n{gages_txt}{seg_txt}{hru_txt}",
    f"\n     {hru_cal_level_txt}\n",
    f"\n{gages_txt_nb2}",
)

In [ ]:
poi_df = create_poi_df(
    model_dir,
    param_filename,
    control_file_name,
    hru_gdf,
    gages_file,
    default_gages_file,
    nwis_gage_nobs_min,
)

In [ ]:
poi_df.isnull().any()

In [ ]:
nwis_gages_aoi[nwis_gages_aoi.poi_id.isnull()]

## Make interactive map of hydrofabric elements: hrus, segments, and gages

In [ ]:
map_file = make_hf_map(
    hru_gdf,
    HW_basins_gdf,
    HW_basins,
    poi_df,
    "",
    seg_gdf,
    nwis_gages_aoi,
    gages_df,
    html_maps_dir,
    param_filename,
    subbasin,
)

map_file_str = f"{map_file}"
_ = webbrowser.open(map_file_str, new=2)

In [ ]:
poi_df.sample(20)

In [ ]:
len(nwis_gages_aoi)